# Solutions to PDEs  #

## 2D Wave Equation ##

The wave equation is 

$$ \nabla^2 \phi = \frac{1}{c^2} \frac{\partial^2\phi}{\partial t}^2 $$

For a rectangular membrane held at $x=0,a$ and $y=0,b$ the solution
separates in rectangular coordinates to give

$$\phi = \phi_0 \sin (k_x x)\sin(k_y y)\cos(\omega t + \delta)$$

with $k_x = n\pi/a$, $k_y = m\pi/b$ and $\omega^2/c^2 = k_x^2+k_y^2$. The arbitrary 
amplitude and phase parameters $\phi_0$ and $\delta$ are of course set by the
initial conditions.

Note: since $\phi(k_x,k_y)$ is a solution to this linear equation, in general
a full solution is of the form $\sum \phi(k_x,k_y)$ i.e. a sum over
components with different wavelengths 
(each with appropriate constants to match the initial 
conditions). Also note that each component has a different
time dependence.

Here we plot the spatial part of $\phi$. 

In [ ]:
# Import maths and plotting modules
from math import *
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from scipy import special as spc

In [ ]:
# Function to plot arbitary excitations on rectangular membrane
def plotRectMembrane(nX,nY):
    # Some parameters (note size only really changes scale on axis)
    sizeX,sizeY = 1,2
    amplitude = 1
    nPoints = 500

    # Calculate wavenumbers
    kx = nX*pi/sizeX
    ky = nY*pi/sizeY

    xPoints = np.linspace(0,sizeX,nPoints)
    yPoints = np.linspace(0,sizeY,nPoints)
    X,Y = np.meshgrid(xPoints,yPoints)
    Z = amplitude*np.sin(kx*X)*np.sin(ky*Y)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(X,Y,Z,cmap=cm.hsv)

    

In [ ]:
# Plot rectangular drum for given coefficients
n = 2
m = 3

plotRectMembrane(n,m)
plt.show()

## Legendre Polynomials and Spherical Harmonics ##

The Legendre polynomials $P_l(x)$ are solutions to Legendre's equation:

$$ (1-x^2)\frac{d^2P_l}{dx^2}-2x\frac{d P_l}{dx}+l(l+1)P_l = 0 $$

and are orthogonal over the range $-1\le x \le 1$. 

They are frequently encountered when solving 3D PDEs by
separating variables in spherical 
coordinates, since the dependence on the polar angle in cases 
with azimuthal symmetry is given
by the above equation with $x = \cos\theta$.

In [ ]:
# Plot Legendre polynomial of order l
def plotLegendre(l):
    nPoints = 500
    xValues = np.linspace(-1,1,nPoints)
    plt.plot(xValues,spc.lpmv(0,l,xValues),label='l='+str(l))
    
plotLegendre(0)
plotLegendre(1)
plotLegendre(2)
plt.grid()
plt.legend()
plt.show()

In the $r-\theta$ plane, the values of $P_l(\cos\theta)$ can be seen to be the potential due to the multpole of order $2^l$ 

as expected from the generating function
$$\frac{1}{\sqrt{1-2hx+h^2}} = \sum_{l=0}^\infty P_l(x) h^l $$

In [ ]:
# Plot multipole potential on polar plot
def plotMultipole(l):
    nPoints = 500

    # Set up coordinate system
    theta = np.linspace(-pi,pi,nPoints)
    r = np.linspace(0,5,100)
    Theta,R = np.meshgrid(theta,r)
    ax = plt.subplot(111, projection='polar')
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)

    # Use masked array to allow masking of points close to singularity
    R = np.ma.array(R)
    R[R<1] = np.ma.masked
    
    # Calculate Legendre function of order l using scipy function: lpmv(0,1,x)
    Z = spc.lpmv(0,l,np.cos(Theta))/np.power(R,l+1)
    mycs = ax.contourf(Theta,R,Z,100,vmin=-1,vmax = 1,cmap = cm.seismic)
    plt.colorbar(mycs)


plotMultipole(2)    
plt.show()

The full angular dependence is in general given by the product of the associated Legendre polynomial with the appropriate azimuthal dependence, known as the spherical harmonic:

$$ Y_l^m(\theta,\phi) \sim P_l^{|m|}(\cos\theta)\, e^{im\theta} $$

with $|m|\le l$

In [ ]:
# Plot desired spherical harmonic as colour plot on a unit sphere
# Copied largely from scipython.com 


def plotSpherical(m,l):
    phi = np.linspace(0, np.pi, 100)
    theta = np.linspace(0, 2*np.pi, 100)
    phi, theta = np.meshgrid(phi, theta)

    # The Cartesian coordinates of the unit sphere
    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)

    # Calculate the spherical harmonic Y(l,m) and normalize to [0,1]
    fcolors = spc.sph_harm(m, l, theta, phi).real
    fmax, fmin = fcolors.max(), fcolors.min()
    fcolors = (fcolors - fmin)/(fmax - fmin)

    # Set the aspect ratio to 1 so our sphere looks spherical
    fig = plt.figure(figsize=plt.figaspect(1.))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x, y, z,rstride=2, cstride=2, facecolors=cm.seismic(fcolors))
    # Turn off the axis planes
    ax.set_axis_off()


plotSpherical(m=1,l=2)

plt.show()


## Bessel Functions ##

In cylindrical coordinates, the radial part of a Laplace-style PDE will typically be Bessel's equation
$$		
		x^2\frac{d^2P}{dx^2}+x\frac{dP}{dx}+\left(x^2-m^2\right)P
		= 0
$$

The solutions of this are known as Bessel functions.

In [ ]:
# Plot a few Bessel functions of the first kind

def plotBessel(n):
    nPoints = 500
    xValues = np.linspace(0,10,nPoints)
    plt.plot(xValues,spc.jv(n,xValues),label='n='+str(n))
    
plotBessel(0)
plotBessel(1)
plotBessel(2)
plt.grid()
plt.legend()
plt.show()

## Computing Exercises ##

1. Experiment with the parameters n and m for the rectangular membrane to see the effect
1. Experiment with the parameters l and m for the Legendre polynomials, multipole expansion and spherical harmonics
1. Plot some Bessel functions of different orders (including negative)
1. Combine the Bessel function with the azimuthal component to plot full solutions for the circular membrane
1. [Advanced] Add the time component to the vibrating membrane examples, to produce an animation showing the full solution for arbitrary modes